# Limpieza de datos

### Importo librerias

In [1]:
import re
import difflib
import numpy as np
import pandas as pd

### Cargo el excel con pandas

In [2]:
full_data = pd.read_excel('./Vecindarios_Alimentarios_Encuesta_Integrada_2022_11_23.xlsx', sheet_name=None)

Obtengo todos los nombres de las hojas del excel

In [3]:
for name in full_data:
    print(name)

Vecindarios Alimentarios - E...
composicion_hogar
_5_18_medidas_control_plagas
_5_19_productos_control_plagas
_5_20_control_malezas
_7_1_Oferta_Alimentos_Produccio
_7_2_produc_dejo_producir_vend
Productos_Vende_Canal
canales_venta_productores
aplicacion_biopreparados
personas_trabajan_puesto_venta
canales_compra_comerciantes
Productos_Compra_Canal
APOYO_RECIBIDO
APOYO_BRINDADO
HUERTO_RELACIONES


dropeo las columnas no relevanntes para este trabajo de limpieza de datos

In [4]:
columns_to_drop = ['_submission__id', '_submission__uuid', '_submission__submission_time', '_submission__validation_status', '_submission__notes', '_submission__status', '_submission__submitted_by', '_submission__tags']

In [5]:
vecindarios = full_data['Vecindarios Alimentarios - E...']
composicion_hogar = full_data['composicion_hogar'].drop(columns=[*columns_to_drop, '_index'])
medidas_control_plagas = full_data['_5_18_medidas_control_plagas'].drop(columns=[*columns_to_drop, '_index'])
productos_control_plagas = full_data['_5_19_productos_control_plagas'].drop(columns=[*columns_to_drop, '_index'])
control_malezas = full_data['_5_20_control_malezas'].drop(columns=[*columns_to_drop, '_index'])
oferta_alimentos = full_data['_7_1_Oferta_Alimentos_Produccio'].drop(columns=[*columns_to_drop, '_index'])
producto_dejo_producir = full_data['_7_2_produc_dejo_producir_vend'].drop(columns=[*columns_to_drop, '_index'])
producto_vende_canal = full_data['Productos_Vende_Canal'].drop(columns=[*columns_to_drop])
canales_venta_productores = full_data['canales_venta_productores'].drop(columns=[*columns_to_drop])
aplicacion_biopreparados = full_data['aplicacion_biopreparados'].drop(columns=[*columns_to_drop, '_index'])
personas_trabajan_puesto_venta = full_data['personas_trabajan_puesto_venta'].drop(columns=[*columns_to_drop, '_index'])
canales_compra_comerciantes = full_data['canales_compra_comerciantes'].drop(columns=[*columns_to_drop])
productos_compra_canal = full_data['Productos_Compra_Canal'].drop(columns=[*columns_to_drop])
APOYO_RECIBIDO = full_data['APOYO_RECIBIDO'].drop(columns=[*columns_to_drop])
APOYO_BRINDADO = full_data['APOYO_BRINDADO'].drop(columns=[*columns_to_drop])
HUERTO_RELACIONES = full_data['HUERTO_RELACIONES'].drop(columns=[*columns_to_drop])

### Composicion_hogar

In [6]:
df_ = composicion_hogar.loc[composicion_hogar['miembro_parentesco']=='el_entrevistado'][['_parent_index','sexo','edad_miembro','miembros_aporta_economicamente']].reset_index(drop=True)

In [7]:
df__ = vecindarios.loc[:,['_index','sexo','edad','aporta_economicamente_hogar']].reset_index(drop=True)

In [8]:
df__['sexo'] = df__.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['sexo'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['sexo'].size != 0 else np.NAN, axis=1)
df__['edad'] = df__.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['edad_miembro'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['edad_miembro'].size != 0 else np.NAN, axis=1)
df__['aporta_economicamente_hogar'] = df__.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['miembros_aporta_economicamente'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['miembros_aporta_economicamente'].size != 0 else np.NAN, axis=1)

In [9]:
vecindarios[['sexo','edad','aporta_economicamente_hogar']] = df__[['sexo','edad','aporta_economicamente_hogar']]

In [10]:
df_ = composicion_hogar.groupby(by=['_parent_index','sexo'], as_index=False).count()[['_parent_index','sexo','miembro_parentesco','otro_pariente','otro_NO_pariente']]
df_['suma'] = df_['miembro_parentesco'] + df_['otro_pariente'] + df_['otro_NO_pariente']

In [11]:
vecindarios['num_mujeres_hogar'] = vecindarios.apply(lambda x: df_.loc[(df_['_parent_index']==x['_index']) & (df_['sexo']=='femenino')]['suma'].iloc[0] if df_.loc[(df_['_parent_index']==x['_index']) & (df_['sexo']=='femenino')]['suma'].size != 0 else 0, axis=1)
vecindarios['num_hombres_hogar'] = vecindarios.apply(lambda x: df_.loc[(df_['_parent_index']==x['_index']) & (df_['sexo']=='masculino')]['suma'].iloc[0] if df_.loc[(df_['_parent_index']==x['_index']) & (df_['sexo']=='masculino')]['suma'].size != 0 else 0, axis=1)

In [12]:
df_ = composicion_hogar.loc[composicion_hogar['edad_miembro'] < 18].groupby(by=['_parent_index'], as_index=False).count()

In [13]:
vecindarios['num_menores_hogar'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['edad_miembro'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['edad_miembro'].size != 0 else 0, axis=1)

In [14]:
df_ = composicion_hogar.loc[composicion_hogar['miembros_aporta_economicamente']=='si'].groupby(by=['_parent_index'], as_index=False).count().rename(columns={'miembros_aporta_economicamente':'aporta_economicamente'})

In [15]:
vecindarios['num_miembros_aportan_hogar'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['aporta_economicamente'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['aporta_economicamente'].size != 0 else 0, axis=1)

In [16]:
df_ = composicion_hogar.drop(columns='_parent_table_name').fillna(value='').astype(str).groupby(by='_parent_index', as_index=False).agg(','.join)
df_ = df_.replace({r'^,+':''}, regex=True)
df_

,_parent_index,miembro_parentesco,otro_pariente,otro_NO_pariente,sexo,edad_miembro,miembros_aporta_economicamente
0,1,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"femenino,masculino,masculino,masculino","54.0,54.0,22.0,25.0","si,si,no,no"
1,10,el_entrevistado,,,femenino,65.0,si
2,100,el_entrevistado,,,masculino,21.0,si
3,101,"el_entrevistado,padres,padres,hijos_hijastros,...",,,"femenino,masculino,femenino,femenino,femenino,...","35.0,38.0,18.0,12.0,17.0","no,si,si,no,no,no"
4,102,"el_entrevistado,otro_pariente,otro_pariente,he...","Sobrino.,Sobrino.,",,"femenino,masculino,masculino,femenino","50.0,8.0,26.0,53.0","si,no,no,si"
...,...,...,...,...,...,...,...
887,95,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"femenino,masculino,masculino,femenino","25.0,24.0,2.0,6.0","si,si,no,no"
888,96,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"femenino,masculino,masculino,femenino","36.0,37.0,6.0,15.0","si,si,no,no"
889,97,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"masculino,femenino,masculino,masculino,masculi...","40.0,39.0,17.0,13.0,6.0,1.0","si,si,si,no,no,no"
890,98,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"femenino,masculino,masculino,masculino,femenino","41.0,43.0,16.0,12.0,10.0","si,no,no,no,no"


In [17]:
for col in df_.columns[1:]:
    if col == 'edad_miembro':
        continue
    df_1 = df_[col].str.get_dummies(sep=',')
    df_1.columns = [col+'/'+i for i in df_1.columns]
    # df_ = df_.drop(columns=col)
    df_ = pd.concat([df_, df_1], axis=1)
df_

,_parent_index,miembro_parentesco,otro_pariente,otro_NO_pariente,sexo,edad_miembro,miembros_aporta_economicamente,miembro_parentesco/el_entrevistado,miembro_parentesco/esposos,miembro_parentesco/hermanos,...,otro_NO_pariente/Conviviente,otro_NO_pariente/Estudiantes,otro_NO_pariente/PADRASTRO,otro_NO_pariente/amigo,otro_NO_pariente/sin parentezco,otro_NO_pariente/trabajador,sexo/femenino,sexo/masculino,miembros_aporta_economicamente/no,miembros_aporta_economicamente/si
0,1,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"femenino,masculino,masculino,masculino","54.0,54.0,22.0,25.0","si,si,no,no",1,1,0,...,0,0,0,0,0,0,1,1,1,1
1,10,el_entrevistado,,,femenino,65.0,si,1,0,0,...,0,0,0,0,0,0,1,0,0,1
2,100,el_entrevistado,,,masculino,21.0,si,1,0,0,...,0,0,0,0,0,0,0,1,0,1
3,101,"el_entrevistado,padres,padres,hijos_hijastros,...",,,"femenino,masculino,femenino,femenino,femenino,...","35.0,38.0,18.0,12.0,17.0","no,si,si,no,no,no",1,0,0,...,0,0,0,0,0,0,1,1,1,1
4,102,"el_entrevistado,otro_pariente,otro_pariente,he...","Sobrino.,Sobrino.,",,"femenino,masculino,masculino,femenino","50.0,8.0,26.0,53.0","si,no,no,si",1,0,1,...,0,0,0,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,95,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"femenino,masculino,masculino,femenino","25.0,24.0,2.0,6.0","si,si,no,no",1,1,0,...,0,0,0,0,0,0,1,1,1,1
888,96,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"femenino,masculino,masculino,femenino","36.0,37.0,6.0,15.0","si,si,no,no",1,1,0,...,0,0,0,0,0,0,1,1,1,1
889,97,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"masculino,femenino,masculino,masculino,masculi...","40.0,39.0,17.0,13.0,6.0,1.0","si,si,si,no,no,no",1,1,0,...,0,0,0,0,0,0,1,1,1,1
890,98,"el_entrevistado,esposos,hijos_hijastros,hijos_...",,,"femenino,masculino,masculino,masculino,femenino","41.0,43.0,16.0,12.0,10.0","si,no,no,no,no",1,1,0,...,0,0,0,0,0,0,1,1,1,1


In [18]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### medidas_control_plagas

In [19]:
df_ = medidas_control_plagas.copy()

In [20]:
df_['_5_18_medida_control_plagas'] = df_['_5_18_medida_control_plagas'].str.lower().str.split(',')
df_ = df_.explode('_5_18_medida_control_plagas')
df_['_5_18_medida_control_plagas'] = df_['_5_18_medida_control_plagas'].str.strip()
df_ = df_.reset_index(drop=True)
df_

,_parent_index,_5_18_medida_control_plagas,_parent_table_name
0,1,aplicación de ceniza,Vecindarios Alimentarios - Encuesta Integrada ...
1,1,aplicación de biopreparados,Vecindarios Alimentarios - Encuesta Integrada ...
2,2,pepas aguacate rayada hervida,Vecindarios Alimentarios - Encuesta Integrada ...
3,2,agua chochos,Vecindarios Alimentarios - Encuesta Integrada ...
4,2,agua cascara de plátano,Vecindarios Alimentarios - Encuesta Integrada ...
...,...,...,...
325,893,lava los tallos de las plantas,Vecindarios Alimentarios - Encuesta Integrada ...
326,893,realiza podas,Vecindarios Alimentarios - Encuesta Integrada ...
327,894,aplica insecticidas,Vecindarios Alimentarios - Encuesta Integrada ...
328,895,usa muy poco remedios,Vecindarios Alimentarios - Encuesta Integrada ...


In [21]:
df_['_5_18_medida_control_plagas'].unique().size

180

In [22]:
# Crea la nueva columna
df_["estandarizado"] = df_["_5_18_medida_control_plagas"].copy()

# Itera sobre cada fila del dataframe
for i in range(len(df_)):
    # Compara la columna específica con todas las demás filas de la misma columna
    match = list(set(difflib.get_close_matches(df_.loc[i, "_5_18_medida_control_plagas"], df_["_5_18_medida_control_plagas"], n=23, cutoff=0.9)))
    # Reemplaza el valor en la columna específica con el valor más similar encontrado
    if match:
        df_.loc[i, "estandarizado"] = match[0]

# Elimina las filas duplicadas
df_ = df_.groupby(by=["_parent_index"], as_index=False).agg(estandar=('estandarizado', ', '.join))
df_

,_parent_index,estandar
0,1,"aplicación de ceniza, aplicación de bio prepar..."
1,2,"pepas aguacate rayada hervida, agua chochos, a..."
2,3,"ceniza, agua canela, vinagre, ají con licor,"
3,4,"agua de cebolla, ají, ajo. purine ajo y ceboll..."
4,5,extractos de hierbas
...,...,...
143,891,aplicación de bioinsumos
144,892,"aplicación de bioinsumos, aplicación de agroq..."
145,893,"aplicacion de paprika, control mecanico, lava ..."
146,894,aplica insecticidas


In [23]:
vecindarios['_5_18_medida_control_plagas'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['estandar'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['estandar'].size != 0 else np.nan, axis=1)

### productos_control_plagas

In [24]:
reemplazar_tildes = {
    'á':'a',
    'é':'e',
    'í':'i',
    'ó':'o',
    'ú':'u'
}

In [25]:
def separar_elementos_por_comas(dataframe: pd.core.frame.DataFrame, columna:str):
    df_ = dataframe.copy()
    df_[columna] = df_[columna].str.lower().str.split(',')
    df_ = df_.explode(columna)
    df_[columna] = df_[columna].str.strip()
    df_ = df_.reset_index(drop=True)
    df_[columna] = df_[columna].replace(to_replace=reemplazar_tildes, regex=True)
    return df_

In [26]:
def estandarizar(dataframe: pd.core.frame.DataFrame, columna:str, n:int=None, cutoff:float=None):
    if not n:
        n = 50
    if not cutoff:
        cutoff = 0.90

    df_ = dataframe.copy()

    # Crea la nueva columna
    df_["estandarizado"] = df_[columna].copy()

    # Itera sobre cada fila del dataframe
    for i in range(len(df_)):
        # Compara la columna específica con todas las demás filas de la misma columna
        match = difflib.get_close_matches(df_.loc[i, columna], df_[columna], n=n, cutoff=cutoff)
        # Reemplaza el valor en la columna específica con el valor más similar encontrado
        if match:
            df_.loc[i, "estandarizado"] = match[0]

    # devolver a valores por comas
    df_ = df_.groupby(by=["_parent_index"], as_index=False).agg(estandar=('estandarizado', ', '.join))
    return df_

In [27]:
productos_control_plagas

,_5_19_producto_control_plagas,_parent_table_name,_parent_index
0,Purín de ají,Vecindarios Alimentarios - Encuesta Integrada ...,1
1,Agua de chochos,Vecindarios Alimentarios - Encuesta Integrada ...,1
2,Preparado de siero de leche,Vecindarios Alimentarios - Encuesta Integrada ...,1
3,Preparado de hoja de tomate riñón,Vecindarios Alimentarios - Encuesta Integrada ...,2
4,"Ají con licor, ceniza,",Vecindarios Alimentarios - Encuesta Integrada ...,3
...,...,...,...
259,Purín de hierbas amargas,Vecindarios Alimentarios - Encuesta Integrada ...,892
260,Caldo sulfocálcico,Vecindarios Alimentarios - Encuesta Integrada ...,892
261,no tiene maleza,Vecindarios Alimentarios - Encuesta Integrada ...,893
262,"furia , lannate",Vecindarios Alimentarios - Encuesta Integrada ...,894


In [28]:
df_ = estandarizar(separar_elementos_por_comas(productos_control_plagas, '_5_19_producto_control_plagas'), '_5_19_producto_control_plagas')
df_

,_parent_index,estandar
0,1,"purin de aji, agua de chochos, preparado de si..."
1,2,preparado de hoja de tomate riñon
2,3,"aji con licor, ceniza,"
3,4,purine de ajo
4,5,"agua de ortiga, purin de ajo, trampas,"
...,...,...
143,891,"caldo sulfocalcico, purin de hierbas amargas, ..."
144,892,"biol zapallo y zanahoria, furalancha, purin de..."
145,893,no tiene maleza
146,894,"furia, lannate"


In [29]:
vecindarios['_5_19_producto_control_plagas'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['estandar'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['estandar'].size != 0 else np.nan, axis=1)

### control_malezas

In [30]:
control_malezas

,_5_20_estrategia_control_malez,_parent_table_name,_parent_index
0,A mano,Vecindarios Alimentarios - Encuesta Integrada ...,1
1,Con herramientas,Vecindarios Alimentarios - Encuesta Integrada ...,1
2,Manual,Vecindarios Alimentarios - Encuesta Integrada ...,2
3,A mano,Vecindarios Alimentarios - Encuesta Integrada ...,3
4,A mano,Vecindarios Alimentarios - Encuesta Integrada ...,4
...,...,...,...
170,Con herramienta,Vecindarios Alimentarios - Encuesta Integrada ...,890
171,Con motoguadaña,Vecindarios Alimentarios - Encuesta Integrada ...,891
172,Con herramienta,Vecindarios Alimentarios - Encuesta Integrada ...,892
173,"herbicida, a mano",Vecindarios Alimentarios - Encuesta Integrada ...,894


In [31]:
df_ = estandarizar(separar_elementos_por_comas(control_malezas, columna='_5_20_estrategia_control_malez'), columna='_5_20_estrategia_control_malez')
df_

,_parent_index,estandar
0,1,"a mano, con herramientas"
1,2,manual
2,3,a mano
3,4,a mano
4,5,a mano
...,...,...
143,890,con herramienta
144,891,con motoguadaña
145,892,con herramienta
146,894,"herbicida, a mano"


In [32]:
vecindarios['_5_20_estrategia_control_malez'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['estandar'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['estandar'].size != 0 else np.nan, axis=1)

### oferta_alimentos

In [33]:
oferta_alimentos

,_parent_index,Producto,Unidad_Cantidad,Precio_de_Venta,Moneda,equivalencia_unidad,porcentaje_perdida_semana,comercializo_2020,motivo_no_comercializ_2020,Utilice_ese_espacio_os_campos_anteriores,_parent_table_name
0,1,Pimientos,25 fundas de 300 gr.,0.60,dolares,7.5 kg.,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
1,1,Tomate Riñón,15 fundas de 1 kilo,1,NaN,15,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
2,1,Brócoli,15 unidades de entre 300 y 400 gr.,0.50,dolares,5.25 kg,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
3,1,Lechuga,20 unidades de 200 gr.,0.50,NaN,4 kg.,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
4,1,Chochos cocinados,15 tarrina de 300 gr.,1,NaN,4.5 kg.,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
...,...,...,...,...,...,...,...,...,...,...,...
576,892,Mora,8 baldes de 13 lb,12,dolares,47.27 kg,NaN,si,No se podía circular,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
577,892,Leche,210 litros,0.47,dolares,NaN,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
578,893,esquejes de pitahaya,esqueje,10,soles,kg,NaN,no,NaN,-,Vecindarios Alimentarios - Encuesta Integrada ...
579,894,girasol,1 paquete son 10 unidades,7-14,soles,kg,NaN,si,no se dedicaba a este cultivo,-,Vecindarios Alimentarios - Encuesta Integrada ...


In [34]:
oferta_alimentos['Producto'] = oferta_alimentos['Producto'].str.lower().str.strip()

In [35]:
df_1 = oferta_alimentos.copy()
df_1['Producto'] = df_1['Producto'].str.split(',')
df_1 = df_1.explode('Producto').reset_index(drop=True)
df_1['Producto'] = df_1['Producto'].str.strip()
df_1 = df_1.fillna(value='')
df_1 = df_1.replace(to_replace=reemplazar_tildes, regex=True)

In [36]:
df_ = df_1.groupby(by='_parent_index', as_index=False).agg(', '.join).replace({r'^,+':np.nan}, regex=True)
df_ = df_.fillna(value='')
df_ = df_.drop(columns=['_parent_table_name'])
df_['Moneda'] = df_['Moneda'].apply(lambda x: re.match(r'^(\w+)', x).group() if re.match(r'^(\w+)', x) else '')
df_

,_parent_index,Producto,Unidad_Cantidad,Precio_de_Venta,Moneda,equivalencia_unidad,porcentaje_perdida_semana,comercializo_2020,motivo_no_comercializ_2020,Utilice_ese_espacio_os_campos_anteriores
0,1,"pimientos, tomate riñon, brocoli, lechuga, cho...","25 fundas de 300 gr., 15 fundas de 1 kilo, 15 ...","0.60, 1, 0.50, 0.50, 1",dolares,"7.5 kg., 15, 5.25 kg, 4 kg., 4.5 kg.",,"no, no, no, no, no",,
1,2,"remolacha, lechuga, rabanos, acelga, suquini, ...","Arroba, Arroba, Arroba, Arroba, Arroba, Arroba...","Na, Na, Na, Na, Na, Na, Na",dolares,"25, 25, 25, 25, 25, 25, 25",,"no, no, no, no, no, no, no",,"Na, Na, Na, Na, Na, Na, Na"
2,3,"acelga, lechuga, rabanos, nabo chino, repollo ...",,,dolares,,,"no, no, no, no, no, no",,
3,4,"hierbas medicinales, cebolla, remolacha, apio,...","No sabe cada socia saca una parte, y se compar...",,dolares,,,"no, no, no, no, no, no, no, no",,"Mas bien se queria ampliar la produccion , Mas..."
4,5,"lechuga, brocoli, coliflor, remolacha, suquini...",,,dolares,,,"no, no, no, no, no, no, no, no, no",,
...,...,...,...,...,...,...,...,...,...,...
233,891,"aguacate, limon, jicama","15 porciones de entre 3/4 unidades, 18 mallas ...","1, 1, 1",dolares,,,"no, no, no",,
234,892,"habas, zuquini, mora, leche","2 sacos de 20 lb, 1 gaveta 15 kg., 8 baldes de...","1 dolar la 1 1/2, 8, 12, 0.47",dolares,"18.18, 15 kg, 47.27 kg,",,"si, no, si, no","No se podia circular, , No se podia circular,",
235,893,esquejes de pitahaya,esqueje,10,soles,kg,,no,,-
236,894,girasol,1 paquete son 10 unidades,7-14,soles,kg,,si,no se dedicaba a este cultivo,-


In [37]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)


### produc_dejo_de_producir_vender

In [38]:
producto_dejo_producir

,otros_productos_n_comerc_2020,motivo_no_comercializo_otros,_parent_table_name,_parent_index
0,Cebolla larga,Falta de mercado,Vecindarios Alimentarios - Encuesta Integrada ...,1
1,NaN,Na,Vecindarios Alimentarios - Encuesta Integrada ...,2
2,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...,3
3,No aplica,NaN,Vecindarios Alimentarios - Encuesta Integrada ...,4
4,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...,5
...,...,...,...,...
169,Hortalizas varias,Se siembra menos porque no hay garantía de venta,Vecindarios Alimentarios - Encuesta Integrada ...,891
170,Papas,Porque llegó la enfermedad de aparatiosis,Vecindarios Alimentarios - Encuesta Integrada ...,892
171,-,-,Vecindarios Alimentarios - Encuesta Integrada ...,893
172,rosas,-,Vecindarios Alimentarios - Encuesta Integrada ...,894


In [39]:
producto_dejo_producir['otros_productos_n_comerc_2020'] = producto_dejo_producir['otros_productos_n_comerc_2020'].str.lower().str.strip()
producto_dejo_producir['motivo_no_comercializo_otros'] = producto_dejo_producir['motivo_no_comercializo_otros'].str.lower().str.strip()
producto_dejo_producir = producto_dejo_producir.drop(columns=['_parent_table_name'])

In [40]:
df_1 = producto_dejo_producir.copy()
df_1['otros_productos_n_comerc_2020'] = df_1['otros_productos_n_comerc_2020'].str.split(',')
df_1 = df_1.explode('otros_productos_n_comerc_2020').reset_index(drop=True)
df_1['otros_productos_n_comerc_2020'] = df_1['otros_productos_n_comerc_2020'].str.strip()
df_1['motivo_no_comercializo_otros'] = df_1['motivo_no_comercializo_otros'].str.strip()
df_1 = df_1.fillna(value='')
df_1 = df_1.replace(to_replace=reemplazar_tildes, regex=True)

In [41]:
df_ = df_1.groupby(by='_parent_index', as_index=False).agg(', '.join).replace({r'^,+':np.nan}, regex=True)
df_ = df_.fillna(value='')
df_ = df_.replace({'-':''}, regex=True)
df_

,_parent_index,otros_productos_n_comerc_2020,motivo_no_comercializo_otros
0,1,cebolla larga,falta de mercado
1,2,,na
2,3,,
3,4,no aplica,
4,5,,
...,...,...,...
157,891,hortalizas varias,se siembra menos porque no hay garantia de venta
158,892,papas,porque llego la enfermedad de aparatiosis
159,893,,
160,894,rosas,


In [42]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### productos_vende_canal

In [43]:
producto_vende_canal

,_parent_index,producto_vendido_canal,nombre_producto_vende,_index,_parent_table_name
0,1,hortalizas___verduras__tubérculos,Pimientos,1,Vecindarios Alimentarios - Encuesta Integrada ...
1,1,hortalizas___verduras__tubérculos,Tomate Riñón,2,Vecindarios Alimentarios - Encuesta Integrada ...
2,1,hortalizas___verduras__tubérculos,Brócoli,3,Vecindarios Alimentarios - Encuesta Integrada ...
3,1,arroz__fideos_y_granos___menestras,Chochos cocinados,4,Vecindarios Alimentarios - Encuesta Integrada ...
4,2,otro_producto,Na,5,Vecindarios Alimentarios - Encuesta Integrada ...
...,...,...,...,...,...
323,892,hortalizas___verduras__tubérculos,Hierbas medicinales,324,Vecindarios Alimentarios - Encuesta Integrada ...
324,893,frutas,esquejes de pitahaya,325,Vecindarios Alimentarios - Encuesta Integrada ...
325,893,frutas,pitahaya,326,Vecindarios Alimentarios - Encuesta Integrada ...
326,894,otro_producto,Girasol,327,Vecindarios Alimentarios - Encuesta Integrada ...


In [44]:
canales_venta_productores

,_index,local_venta_productores,nombre,motivos,transporte_vende_productos,distancia,canal_principal_2019,canal_principal_2020,_parent_table_name,_parent_index
0,1,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializó o intercambio,Productos_Vende_Canal,1
1,2,bioferias,Feria Casa Mitómana,Porque está disponibles para los agricultores,Bus y taxi,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio,Productos_Vende_Canal,1
2,3,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio,Productos_Vende_Canal,2
3,4,bioferias,Feria Casa Mitómana,Porque está disponibles para los agricultores,Bus y taxi,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio,Productos_Vende_Canal,2
4,5,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio,Productos_Vende_Canal,3
...,...,...,...,...,...,...,...,...,...,...
400,401,otro_canal,Mercado Mayorista de Ambato,Porque hay demanda,Camioneta propia,de_5_a_10_km,Mercado Mayorista de Ambato,Mercado Mayorista de Ambato,Productos_Vende_Canal,324
401,402,otro_canal,"huerto, facebook",facilidad de venta,-,NaN,-,-,Productos_Vende_Canal,325
402,403,otro_canal,consumo propio,-,-,NaN,-,-,Productos_Vende_Canal,326
403,404,otro_canal,Mercado Santa Rosa,venta de flores,camioneta,de_5_a_10_km,no sembraba girasoles,no sembraba girasoles,Productos_Vende_Canal,327


In [45]:
df_ = pd.merge(left=producto_vende_canal, right=canales_venta_productores, left_on='_index', right_on='_parent_index', suffixes=('','_y'))
df_ = df_.drop(columns=[i for i in df_.columns if i.endswith('_y')])
df_ = df_.fillna(value='')
df_ = df_.drop_duplicates(subset=['_parent_index','_index','nombre_producto_vende'])
df_

,_parent_index,producto_vendido_canal,nombre_producto_vende,_index,_parent_table_name,local_venta_productores,nombre,motivos,transporte_vende_productos,distancia,canal_principal_2019,canal_principal_2020
0,1,hortalizas___verduras__tubérculos,Pimientos,1,Vecindarios Alimentarios - Encuesta Integrada ...,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializó o intercambio
2,1,hortalizas___verduras__tubérculos,Tomate Riñón,2,Vecindarios Alimentarios - Encuesta Integrada ...,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio
4,1,hortalizas___verduras__tubérculos,Brócoli,3,Vecindarios Alimentarios - Encuesta Integrada ...,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio
6,1,arroz__fideos_y_granos___menestras,Chochos cocinados,4,Vecindarios Alimentarios - Encuesta Integrada ...,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio
8,2,otro_producto,Na,5,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,Na,Na,Na,no_aplica,Na,Na
...,...,...,...,...,...,...,...,...,...,...,...,...
400,892,hortalizas___verduras__tubérculos,Hierbas medicinales,324,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,Mercado Mayorista de Ambato,Porque hay demanda,Camioneta propia,de_5_a_10_km,Mercado Mayorista de Ambato,Mercado Mayorista de Ambato
401,893,frutas,esquejes de pitahaya,325,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,"huerto, facebook",facilidad de venta,-,,-,-
402,893,frutas,pitahaya,326,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,consumo propio,-,-,,-,-
403,894,otro_producto,Girasol,327,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,Mercado Santa Rosa,venta de flores,camioneta,de_5_a_10_km,no sembraba girasoles,no sembraba girasoles


In [46]:
df_['producto_vendido_canal'] = df_['producto_vendido_canal'].str.lower().str.strip()
df_['nombre_producto_vende'] = df_['nombre_producto_vende'].str.lower().str.strip()
df_['local_venta_productores'] = df_['local_venta_productores'].str.lower().str.strip()
df_['nombre'] = df_['nombre'].str.lower().str.strip()
df_['motivos'] = df_['motivos'].str.lower().str.strip()
df_['transporte_vende_productos'] = df_['transporte_vende_productos'].str.lower().str.strip()
df_['distancia'] = df_['distancia'].str.lower().str.strip()
df_['canal_principal_2019'] = df_['canal_principal_2019'].str.lower().str.strip()
df_['canal_principal_2020'] = df_['canal_principal_2020'].str.lower().str.strip()

In [47]:
df_ = df_.replace(to_replace=reemplazar_tildes|{'-':''}, regex=True)

In [48]:
df__ = df_.groupby(by='_parent_index', as_index=False).agg(', '.join).replace({r'^,+':np.nan}, regex=True).fillna(value='')
df__

c:\Users\norte\anaconda3\envs\hackspace_portal\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,_parent_index,producto_vendido_canal,nombre_producto_vende,_parent_table_name,local_venta_productores,nombre,motivos,transporte_vende_productos,distancia,canal_principal_2019,canal_principal_2020
0,1,"hortalizas___verduras__tuberculos, hortalizas_...","pimientos, tomate riñon, brocoli, chochos coci...",Vecindarios Alimentarios Encuesta Integrada (...,"bioferias, bioferias, bioferias, bioferias","feria la carolina, feria la carolina, feria la...","porque esta disponibles para los agricultores,...","camioneta de alquiler, camioneta de alquiler, ...","mas_de_10_km, mas_de_10_km, mas_de_10_km, mas_...","feria la carolina, feria la carolina, feria la...","no se comercializo o intercambio, no se comerc..."
1,2,otro_producto,na,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,na,na,na,no_aplica,na,na
2,3,otro_producto,,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,,,,no_aplica,,
3,4,otro_producto,,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,,,,no_aplica,,
4,5,otro_producto,,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,,,,no_aplica,,
...,...,...,...,...,...,...,...,...,...,...,...
142,891,"frutas, frutas, hortalizas___verduras__tuberculos","aguacate, limon, verduras y tuberculos variados",Vecindarios Alimentarios Encuesta Integrada (...,"bioferias, bioferias, bioferias","feria de carcelen, feria de carcelen, feria de...","porque esta disponible, porque esta disponible...","bus y/o camioneta alquilada, bus y/o camioneta...","mas_de_10_km, mas_de_10_km, mas_de_10_km","feria de carcelen, feria de carcelen, feria de...","no se comercializaba, no se comercializaba, no..."
143,892,"leche_y_productos_lacteos, hortalizas___verdur...","leche, zuquini, habas, hortalizas variadas, hi...",Vecindarios Alimentarios Encuesta Integrada (...,"otro_canal, bioferias, otro_canal, bioferias, ...","empresa indulac, mercado madre tierra, mercado...","porque hay demanda, porque esta disponible y s...","recoleccion a pie de finca, camioneta propia, ...","no_aplica, mas_de_10_km, de_5_a_10_km, mas_de_...","intermediario, mercado madre tierra, mercado m...","intermediario, no se comercializo, no se comer..."
144,893,"frutas, frutas","esquejes de pitahaya, pitahaya",Vecindarios Alimentarios Encuesta Integrada (...,"otro_canal, otro_canal","huerto, facebook, consumo propio","facilidad de venta,",,,,
145,894,otro_producto,girasol,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,mercado santa rosa,venta de flores,camioneta,de_5_a_10_km,no sembraba girasoles,no sembraba girasoles


In [49]:
df_1 = df__['producto_vendido_canal'].str.get_dummies(sep=', ')
df_1.columns = ['producto_vendido_canal/'+i for i in df_1.columns]
df_2 = df__['local_venta_productores'].str.get_dummies(sep=', ')
df_2.columns = ['local_venta_productores/'+i for i in df_2.columns]
df_3 = df__['distancia'].str.get_dummies(sep=', ')
df_3.columns = ['distancia/'+i for i in df_3.columns]
df__ = df__.drop(columns=['producto_vendido_canal', 'local_venta_productores', 'distancia'])
df__ = pd.concat([df__, df_1, df_2, df_3], axis=1)
df__

,_parent_index,nombre_producto_vende,_parent_table_name,nombre,motivos,transporte_vende_productos,canal_principal_2019,canal_principal_2020,producto_vendido_canal/arroz__fideos_y_granos___menestras,producto_vendido_canal/carnes_y_pescados,...,local_venta_productores/intermediarios,local_venta_productores/mayoristas,local_venta_productores/mercados_de_abasto,local_venta_productores/otro_canal,local_venta_productores/tiendas_especializadas__organicos_o_agroecologicos_,distancia/de_1_a_5_km,distancia/de_5_a_10_km,distancia/mas_de_10_km,distancia/menor_a_1_km,distancia/no_aplica
0,1,"pimientos, tomate riñon, brocoli, chochos coci...",Vecindarios Alimentarios Encuesta Integrada (...,"feria la carolina, feria la carolina, feria la...","porque esta disponibles para los agricultores,...","camioneta de alquiler, camioneta de alquiler, ...","feria la carolina, feria la carolina, feria la...","no se comercializo o intercambio, no se comerc...",1,0,...,0,0,0,0,0,0,0,1,0,0
1,2,na,Vecindarios Alimentarios Encuesta Integrada (...,na,na,na,na,na,0,0,...,0,0,0,1,0,0,0,0,0,1
2,3,,Vecindarios Alimentarios Encuesta Integrada (...,,,,,,0,0,...,0,0,0,1,0,0,0,0,0,1
3,4,,Vecindarios Alimentarios Encuesta Integrada (...,,,,,,0,0,...,0,0,0,1,0,0,0,0,0,1
4,5,,Vecindarios Alimentarios Encuesta Integrada (...,,,,,,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,891,"aguacate, limon, verduras y tuberculos variados",Vecindarios Alimentarios Encuesta Integrada (...,"feria de carcelen, feria de carcelen, feria de...","porque esta disponible, porque esta disponible...","bus y/o camioneta alquilada, bus y/o camioneta...","feria de carcelen, feria de carcelen, feria de...","no se comercializaba, no se comercializaba, no...",0,0,...,0,0,0,0,0,0,0,1,0,0
143,892,"leche, zuquini, habas, hortalizas variadas, hi...",Vecindarios Alimentarios Encuesta Integrada (...,"empresa indulac, mercado madre tierra, mercado...","porque hay demanda, porque esta disponible y s...","recoleccion a pie de finca, camioneta propia, ...","intermediario, mercado madre tierra, mercado m...","intermediario, no se comercializo, no se comer...",1,0,...,0,0,0,1,0,0,1,1,0,1
144,893,"esquejes de pitahaya, pitahaya",Vecindarios Alimentarios Encuesta Integrada (...,"huerto, facebook, consumo propio","facilidad de venta,",,,,0,0,...,0,0,0,1,0,0,0,0,0,0
145,894,girasol,Vecindarios Alimentarios Encuesta Integrada (...,mercado santa rosa,venta de flores,camioneta,no sembraba girasoles,no sembraba girasoles,0,0,...,0,0,0,1,0,0,1,0,0,0


In [50]:
df_ = df__.copy()

In [51]:
df_.drop(columns=['_parent_table_name'], inplace=True)

In [52]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### aplicacion_biopreparados

In [53]:
aplicacion_biopreparados = aplicacion_biopreparados.fillna(value='')
aplicacion_biopreparados = aplicacion_biopreparados.drop(columns=['_parent_table_name'])
aplicacion_biopreparados

,_10_2_Quien_aplica_biopreparad,sexo_contratado,parentesco_familiar,Indique_otro_pariente,otra_persona_aplica_bioprep,_10_2_1_usa_proteccion,_parent_index
0,yo_mismo,,,,,no,1
1,pariente_s,,esposos,,,no,1
2,yo_mismo,,,,,no,11
3,pariente_s,,hijos_hijastros,,,no,11
4,pariente_s,,hijos_hijastros,,,parcial,12
...,...,...,...,...,...,...,...
123,yo_mismo,,,,,no,891
124,pariente_s,,esposos,,,parcial,892
125,yo_mismo,,,,,parcial,893
126,yo_mismo,,,,,no,894


In [54]:
columnas = ['_10_2_Quien_aplica_biopreparad','sexo_contratado','parentesco_familiar','Indique_otro_pariente','otra_persona_aplica_bioprep','_10_2_1_usa_proteccion']

In [55]:
df_ = aplicacion_biopreparados.copy()
df_ = pd.get_dummies(df_, columns=columnas, prefix=columnas, prefix_sep='/')
df_

,_parent_index,_10_2_Quien_aplica_biopreparad/contratado_s,_10_2_Quien_aplica_biopreparad/otra_persona,_10_2_Quien_aplica_biopreparad/pariente_s,_10_2_Quien_aplica_biopreparad/yo_mismo,sexo_contratado/,sexo_contratado/hombre,sexo_contratado/mujer,parentesco_familiar/,parentesco_familiar/esposos,...,parentesco_familiar/padres,parentesco_familiar/yerno_nuera,Indique_otro_pariente/,Indique_otro_pariente/cuñado,otra_persona_aplica_bioprep/,otra_persona_aplica_bioprep/Hijo,_10_2_1_usa_proteccion/,_10_2_1_usa_proteccion/no,_10_2_1_usa_proteccion/parcial,_10_2_1_usa_proteccion/si
0,1,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
1,1,0,0,1,0,1,0,0,0,1,...,0,0,1,0,1,0,0,1,0,0
2,11,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
3,11,0,0,1,0,1,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
4,12,0,0,1,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,891,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
124,892,0,0,1,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
125,893,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,0,1,0
126,894,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0


In [56]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### personas_trabajan_puesto_de_venta

In [57]:
df_ = personas_trabajan_puesto_venta.copy()
df_.drop(columns=['_parent_table_name','otro_pariente_empleado','otra_persona_empleada',*[i for i in df_.columns if i.startswith('Indica')]], inplace=True)
df_['_b_horas_porsemana_trabaja'] = df_['_b_horas_porsemana_trabaja'].fillna(value='')
df_['_b_horas_porsemana_trabaja'] = df_['_b_horas_porsemana_trabaja'].apply(lambda x: re.match(r'\d+', x).group() if re.match(r'\d+', x) else 'esporadico')
df_

,personas_empleadas_puesto,pariente_empleado,_b_horas_porsemana_trabaja,_c_persona_recibe_remuneracion,_parent_index
0,pariente,esposos,8,no,24
1,empleado,NaN,7,no,48
2,pariente,esposos,8,no,50
3,pariente,esposos,8,si,53
4,pariente,esposos,8,no,55
5,pariente,esposos,8,si,56
6,pariente,hijos_hijastros,esporadico,no,74
7,pariente,hijos_hijastros,7,si,107
8,empleado,NaN,10,si,114
9,empleado,NaN,10,si,114


In [58]:
columns = ['personas_empleadas_puesto','pariente_empleado','_b_horas_porsemana_trabaja','_c_persona_recibe_remuneracion']

In [59]:
df_ = pd.get_dummies(df_, columns=columns, prefix=columns, prefix_sep='/')

In [60]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### canales_compra_comerciantes

In [61]:
df_ = canales_compra_comerciantes.copy()
df_ = df_.drop(columns=['_parent_table_name'])
df_['local_compra_comerciantes'] = df_['local_compra_comerciantes'].str.strip().str.lower().replace(to_replace=reemplazar_tildes, regex=True)
df_['nombre_local_compra'] = df_['nombre_local_compra'].str.strip().str.lower().replace(to_replace=reemplazar_tildes, regex=True)
df_['frecuencia_compra'] = df_['frecuencia_compra'].str.strip().str.lower().replace(to_replace=reemplazar_tildes, regex=True)
df_['motivos_compra_canal'] = df_['motivos_compra_canal'].str.strip().str.lower().replace(to_replace=reemplazar_tildes, regex=True)
df_['transporte_compra_productos'] = df_['transporte_compra_productos'].str.strip().str.lower().replace(to_replace=reemplazar_tildes, regex=True)
df_['_2019_canal_utilizado'] = df_['_2019_canal_utilizado'].str.strip().str.lower().replace(to_replace=reemplazar_tildes, regex=True)
df_['_2020_canal_utilizado'] = df_['_2020_canal_utilizado'].str.strip().str.lower().replace(to_replace=reemplazar_tildes, regex=True)
df_

,local_compra_comerciantes,nombre_local_compra,frecuencia_compra,motivos_compra_canal,transporte_compra_productos,_2019_canal_utilizado,_2020_canal_utilizado,_index,_parent_index
0,mercados_de_abasto,unicachi,1_vez_a_la_semana,mas variedad,taxi,unicachi,unicachi,1,24
1,mercados_de_abasto,mercado mayorista,2_veces_a_la_semana,variedad,colectivo,colectivo,colectivo,2,44
2,mercados_de_abasto,mercado modelo,1_vez_a_la_semana,cercania,carro,mercado modelo,mercado modelo,3,45
3,mercados_de_abasto,unicachi,1_vez_a_la_semana,cercania,carro,unicachi,unicachi,4,45
4,intermediarios,NaN,NaN,NaN,NaN,NaN,NaN,5,47
...,...,...,...,...,...,...,...,...,...
99,mercados_de_abasto,mayorista,2_veces_a_la_semana,cercania al local,caminando,mayorista,mayorista,100,819
100,NaN,mayorista,2_veces_a_la_semana,comodidad y economia,camioneta,mayorista,mayorista y san roque,101,820
101,mercados_de_abasto,mercado america,2_veces_a_la_semana,unicos lugares,camioneta,mercado america,no compro,102,821
102,mercados_de_abasto,san roque,2_veces_a_la_semana,unicos lugares,camioneta,san roque,no compro,103,821


In [62]:
df__ = productos_compra_canal.copy()
df__ = df__.drop(columns=['_parent_table_name','_index'])
df__['nombre_producto_compra'] = df__['nombre_producto_compra'].str.lower().str.strip().replace(to_replace=reemplazar_tildes, regex=True)
df__

,producto_comprado_canal,nombre_producto_compra,_parent_index
0,frutas,"mandarina platanos, uvas",1
1,frutas,"mandarina, tomate y papa",2
2,frutas,NaN,3
3,hortalizas___verduras__tubérculos,NaN,4
4,frutas,platano,6
...,...,...,...
125,frutas,platanos,99
126,hortalizas___verduras__tubérculos,alverja,100
127,hortalizas___verduras__tubérculos,"papas, zanahorias, alverja, choclo, garbanzo",101
128,otro_producto,flores,102


In [63]:
df_a = pd.merge(left=df_, right=df__, left_on='_index', right_on='_parent_index', suffixes=('','_y'))
df_a = df_a.drop(columns=[i for i in df_a.columns if i.endswith('_y')])
df_a = df_a.fillna(value='')
df_a = df_a.drop_duplicates(subset=['_parent_index','_index'])
df_a

,local_compra_comerciantes,nombre_local_compra,frecuencia_compra,motivos_compra_canal,transporte_compra_productos,_2019_canal_utilizado,_2020_canal_utilizado,_index,_parent_index,producto_comprado_canal,nombre_producto_compra
0,mercados_de_abasto,unicachi,1_vez_a_la_semana,mas variedad,taxi,unicachi,unicachi,1,24,frutas,"mandarina platanos, uvas"
1,mercados_de_abasto,mercado mayorista,2_veces_a_la_semana,variedad,colectivo,colectivo,colectivo,2,44,frutas,"mandarina, tomate y papa"
2,mercados_de_abasto,mercado modelo,1_vez_a_la_semana,cercania,carro,mercado modelo,mercado modelo,3,45,frutas,
3,mercados_de_abasto,unicachi,1_vez_a_la_semana,cercania,carro,unicachi,unicachi,4,45,hortalizas___verduras__tubérculos,
4,mercados_de_abasto,mercado mayorista,3_veces_a_la_semana,variedad,colectivo,colectivo,colectivo,6,48,frutas,platano
...,...,...,...,...,...,...,...,...,...,...,...
125,mercados_de_abasto,mayorista,2_veces_a_la_semana,productores y economia,camioneta,mayorista,mayorista,99,817,frutas,platanos
126,mercados_de_abasto,mayorista,2_veces_a_la_semana,cercania al local,caminando,mayorista,mayorista,100,819,hortalizas___verduras__tubérculos,alverja
127,,mayorista,2_veces_a_la_semana,comodidad y economia,camioneta,mayorista,mayorista y san roque,101,820,hortalizas___verduras__tubérculos,"papas, zanahorias, alverja, choclo, garbanzo"
128,mercados_de_abasto,mercado america,2_veces_a_la_semana,unicos lugares,camioneta,mercado america,no compro,102,821,otro_producto,flores


In [64]:
df_a = df_a.replace(to_replace=reemplazar_tildes|{'-':''}, regex=True)

In [65]:
df__ = df_a.groupby(by='_parent_index', as_index=False).agg(', '.join).replace({r'^,+':np.nan}, regex=True).fillna(value='')
df__

c:\Users\norte\anaconda3\envs\hackspace_portal\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,_parent_index,local_compra_comerciantes,nombre_local_compra,frecuencia_compra,motivos_compra_canal,transporte_compra_productos,_2019_canal_utilizado,_2020_canal_utilizado,producto_comprado_canal,nombre_producto_compra
0,24,mercados_de_abasto,unicachi,1_vez_a_la_semana,mas variedad,taxi,unicachi,unicachi,frutas,"mandarina platanos, uvas"
1,44,mercados_de_abasto,mercado mayorista,2_veces_a_la_semana,variedad,colectivo,colectivo,colectivo,frutas,"mandarina, tomate y papa"
2,45,"mercados_de_abasto, mercados_de_abasto","mercado modelo, unicachi","1_vez_a_la_semana, 1_vez_a_la_semana","cercania, cercania","carro, carro","mercado modelo, unicachi","mercado modelo, unicachi","frutas, hortalizas___verduras__tuberculos",
3,48,mercados_de_abasto,mercado mayorista,3_veces_a_la_semana,variedad,colectivo,colectivo,colectivo,frutas,platano
4,49,mayoristas,valle del chillon,3_veces_a_la_semana,cerca,mototaxi,mototaxi,mototaxi,hortalizas___verduras__tuberculos,hortalizas
...,...,...,...,...,...,...,...,...,...,...
82,817,mercados_de_abasto,mayorista,2_veces_a_la_semana,productores y economia,camioneta,mayorista,mayorista,frutas,platanos
83,819,mercados_de_abasto,mayorista,2_veces_a_la_semana,cercania al local,caminando,mayorista,mayorista,hortalizas___verduras__tuberculos,alverja
84,820,,mayorista,2_veces_a_la_semana,comodidad y economia,camioneta,mayorista,mayorista y san roque,hortalizas___verduras__tuberculos,"papas, zanahorias, alverja, choclo, garbanzo"
85,821,mercados_de_abasto,mercado america,2_veces_a_la_semana,unicos lugares,camioneta,mercado america,no compro,otro_producto,flores


In [66]:
df_1 = df__['local_compra_comerciantes'].str.get_dummies(sep=', ')
df_1.columns = ['local_compra_comerciantes/'+i for i in df_1.columns]
df_2 = df__['frecuencia_compra'].str.get_dummies(sep=', ')
df_2.columns = ['frecuencia_compra/'+i for i in df_2.columns]
df_3 = df__['producto_comprado_canal'].str.get_dummies(sep=', ')
df_3.columns = ['producto_comprado_canal/'+i for i in df_3.columns]
df__ = df__.drop(columns=['local_compra_comerciantes', 'frecuencia_compra', 'producto_comprado_canal'])
df__ = pd.concat([df__, df_1, df_2, df_3], axis=1)
df__['nombre_local_compra'] = df__['nombre_local_compra'].replace({r'\bcerca\b': 'cercania', r'\bamerica\b': 'americas'}, regex=True)
df__['motivos_compra_canal'] = df__['motivos_compra_canal'].replace({r'\bcerca\b': 'cercania', r'\bamerica\b': 'americas'}, regex=True)
df__

,_parent_index,nombre_local_compra,motivos_compra_canal,transporte_compra_productos,_2019_canal_utilizado,_2020_canal_utilizado,nombre_producto_compra,local_compra_comerciantes/intermediarios,local_compra_comerciantes/mayoristas,local_compra_comerciantes/mercados_de_abasto,...,frecuencia_compra/3_veces_a_la_semana,frecuencia_compra/4_veces_a_la_semana,frecuencia_compra/todos_los_dias,producto_comprado_canal/arroz__fideos_y_granos___menestras,producto_comprado_canal/carnes_y_pescados,producto_comprado_canal/frutas,producto_comprado_canal/hortalizas___verduras__tuberculos,producto_comprado_canal/huevos,producto_comprado_canal/leche_y_productos_lacteos,producto_comprado_canal/otro_producto
0,24,unicachi,mas variedad,taxi,unicachi,unicachi,"mandarina platanos, uvas",0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,44,mercado mayorista,variedad,colectivo,colectivo,colectivo,"mandarina, tomate y papa",0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,45,"mercado modelo, unicachi","cercania, cercania","carro, carro","mercado modelo, unicachi","mercado modelo, unicachi",,0,0,1,...,0,0,0,0,0,1,1,0,0,0
3,48,mercado mayorista,variedad,colectivo,colectivo,colectivo,platano,0,0,1,...,1,0,0,0,0,1,0,0,0,0
4,49,valle del chillon,cercania,mototaxi,mototaxi,mototaxi,hortalizas,0,1,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,817,mayorista,productores y economia,camioneta,mayorista,mayorista,platanos,0,0,1,...,0,0,0,0,0,1,0,0,0,0
83,819,mayorista,cercania al local,caminando,mayorista,mayorista,alverja,0,0,1,...,0,0,0,0,0,0,1,0,0,0
84,820,mayorista,comodidad y economia,camioneta,mayorista,mayorista y san roque,"papas, zanahorias, alverja, choclo, garbanzo",0,0,0,...,0,0,0,0,0,0,1,0,0,0
85,821,mercado americas,unicos lugares,camioneta,mercado america,no compro,flores,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [67]:
df_ = df__.copy()

In [68]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### APOYO_RECIBIDO

In [69]:
APOYO_RECIBIDO

,_a_de_quien_recibio_apoyo,personas_recibio_apoyo,otras_personas_recibio_apoyo,parentesco_recibio_apoyo,otros_parientes_recibio_apoyo,grupo_recibio_apoyo,organizacion_recibio_apoyo,_b_Sexo_recibio_apoyo,Nombre_grupo_organiz_recibio,_c_vecindario_recibio_apoyo,...,_d_tipo_apoyo_recibido/dinero,_d_tipo_apoyo_recibido/alimentos,_d_tipo_apoyo_recibido/informaciones,_d_tipo_apoyo_recibido/otro_apoyo,otro_tipo_apoyo_recibido,_e_frecuencia_recibio_apoyo,_f_Actualmente_recibe_apoyo,_index,_parent_table_name,_parent_index
0,organizaciones,NaN,NaN,NaN,NaN,NaN,centros_educacion,NaN,colegio,si,...,0.0,1.0,0.0,0.0,NaN,algunas_veces,no,440,Vecindarios Alimentarios - Encuesta Integrada ...,697
1,organizaciones,NaN,NaN,NaN,NaN,NaN,centros_educacion,NaN,Colegio Raymondi,no,...,1.0,0.0,0.0,0.0,NaN,algunas_veces,no,470,Vecindarios Alimentarios - Encuesta Integrada ...,755
2,organizaciones,NaN,NaN,NaN,NaN,NaN,empresa,NaN,Minisa,no,...,1.0,0.0,0.0,0.0,NaN,una_vez,no,309,Vecindarios Alimentarios - Encuesta Integrada ...,446
3,organizaciones,NaN,NaN,NaN,NaN,NaN,empresa,NaN,Su trabajo,no,...,1.0,1.0,0.0,0.0,NaN,algunas_veces,no,362,Vecindarios Alimentarios - Encuesta Integrada ...,545
4,organizaciones,NaN,NaN,NaN,NaN,NaN,gobierno,NaN,AFP,NaN,...,1.0,0.0,0.0,0.0,NaN,una_vez,no,449,Vecindarios Alimentarios - Encuesta Integrada ...,721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,personas,vecino,NaN,NaN,NaN,NaN,NaN,mujer,NaN,si,...,0.0,1.0,0.0,1.0,Insumos,algunas_veces,no,563,Vecindarios Alimentarios - Encuesta Integrada ...,865
581,personas,amigo,NaN,NaN,NaN,NaN,NaN,hombre,NaN,si,...,0.0,1.0,0.0,1.0,Insumo,algunas_veces,no,564,Vecindarios Alimentarios - Encuesta Integrada ...,866
582,personas,vecino,NaN,NaN,NaN,NaN,NaN,hombre,NaN,si,...,0.0,0.0,0.0,1.0,"Insumo, apoyo técnico",algunas_veces,si,566,Vecindarios Alimentarios - Encuesta Integrada ...,869
583,personas,vecino,NaN,NaN,NaN,NaN,NaN,mujer,NaN,si,...,0.0,0.0,0.0,1.0,NaN,algunas_veces,si,567,Vecindarios Alimentarios - Encuesta Integrada ...,870


In [70]:
df_ = APOYO_RECIBIDO.copy()
df_ = df_.fillna(value='')
df_ = df_.astype(str)
df_ = df_.apply(lambda x: x.str.lower(), axis=1)

In [71]:
df_ = df_.fillna(value='').groupby(by='_parent_index', as_index=False).agg(','.join)
df_['personas_recibio_apoyo'] = df_['personas_recibio_apoyo'].replace({r'^,+':''}, regex=True)
df_['otras_personas_recibio_apoyo'] = df_['otras_personas_recibio_apoyo'].replace({r'^,+':''}, regex=True)
df_['parentesco_recibio_apoyo'] = df_['parentesco_recibio_apoyo'].replace({r'^,+':''}, regex=True)
df_['otros_parientes_recibio_apoyo'] = df_['otros_parientes_recibio_apoyo'].replace({r'^,+':''}, regex=True)
df_['grupo_recibio_apoyo'] = df_['grupo_recibio_apoyo'].replace({r'^,+':''}, regex=True)
df_['organizacion_recibio_apoyo'] = df_['organizacion_recibio_apoyo'].replace({r'^,+':''}, regex=True)
df_['_b_Sexo_recibio_apoyo'] = df_['_b_Sexo_recibio_apoyo'].replace({r'^,+':''}, regex=True)
df_['Nombre_grupo_organiz_recibio'] = df_['Nombre_grupo_organiz_recibio'].replace({r'^,+':''}, regex=True)
df_['otro_tipo_apoyo_recibido'] = df_['otro_tipo_apoyo_recibido'].replace({r'^,+':''}, regex=True)
df_['_e_frecuencia_recibio_apoyo'] = df_['_e_frecuencia_recibio_apoyo'].replace({r'^,+':''}, regex=True)
df_['_f_Actualmente_recibe_apoyo'] = df_['_f_Actualmente_recibe_apoyo'].replace({r'^,+':''}, regex=True)
df_ = df_.drop(columns=['_parent_table_name','_index'])

In [72]:
df_

,_parent_index,_a_de_quien_recibio_apoyo,personas_recibio_apoyo,otras_personas_recibio_apoyo,parentesco_recibio_apoyo,otros_parientes_recibio_apoyo,grupo_recibio_apoyo,organizacion_recibio_apoyo,_b_Sexo_recibio_apoyo,Nombre_grupo_organiz_recibio,_c_vecindario_recibio_apoyo,_d_tipo_apoyo_recibido,_d_tipo_apoyo_recibido/dinero,_d_tipo_apoyo_recibido/alimentos,_d_tipo_apoyo_recibido/informaciones,_d_tipo_apoyo_recibido/otro_apoyo,otro_tipo_apoyo_recibido,_e_frecuencia_recibio_apoyo,_f_Actualmente_recibe_apoyo
0,10,organizaciones,,,,,,gobierno,,conquito,no,otro_apoyo,0.0,0.0,0.0,1.0,apoyo técnico,frecuentemente,si
1,100,"personas,personas","familia,familia",,"padres,padres",,,,"hombre,mujer",,"no,no","dinero,alimentos","1.0,0.0","0.0,1.0","0.0,0.0","0.0,0.0",,"frecuentemente,frecuentemente","no,no"
2,106,"personas,personas","otra_persona,familia","cuñada,",otro_pariente,sobrina,,,"mujer,mujer",,"no,no","alimentos,alimentos","0.0,0.0","1.0,1.0","0.0,0.0","0.0,0.0",,"una_vez,algunas_veces","no,no"
3,109,personas,familia,,hermanos,,,,hombre,,si,alimentos,0.0,1.0,0.0,0.0,,algunas_veces,no
4,110,personas,familia,,hermanos,,,,mujer,,si,dinero alimentos,1.0,1.0,0.0,0.0,,frecuentemente,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,91,organizaciones,,,,,,gobierno,,,no,dinero,1.0,0.0,0.0,0.0,,algunas_veces,no
412,93,organizaciones,,,,,,gobierno,,,no,dinero,1.0,0.0,0.0,0.0,,algunas_veces,no
413,94,organizaciones,,,,,,gobierno,,,no,dinero,1.0,0.0,0.0,0.0,,una_vez,no
414,97,"organizaciones,personas",familia,,otro_pariente,tío,,"gobierno,",hombre,,"no,si","dinero,alimentos","1.0,0.0","0.0,1.0","0.0,0.0","0.0,0.0",,"una_vez,una_vez","no,no"


In [73]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### apoyo_brindado

In [74]:
df_ = APOYO_BRINDADO.copy()
df_ = df_.fillna(value='')
df_ = df_.astype(str)
df_ = df_.apply(lambda x: x.str.lower(), axis=1)

In [75]:
df_ = df_.groupby(by='_parent_index', as_index=False).agg(','.join).replace({r'^,+':''}, regex=True).drop(columns=['_parent_table_name','_index'])

In [76]:
df_

,_parent_index,_a_quien_brindo_apoyo,personas_brindo_apoyo,otras_personas_brindo_apoyo,parentesco_brindo_apoyo,otros_parientes_brindo_apoyo,grupo_brindo_apoyo,organizacion_brindo_apoyo,_b_Sexo_brindo_apoyo,Nombre_grupo_organiz_brindo,_c_vecindario_brindo_apoyo,_d_tipo_apoyo_brindado,_d_tipo_apoyo_brindado/dinero,_d_tipo_apoyo_brindado/alimentos,_d_tipo_apoyo_brindado/informaciones,_d_tipo_apoyo_brindado/otro_apoyo,otro_tipo_apoyo_brindado,_e_frecuencia_brindo_apoyo,_f_Actualmente_brinda_apoyo
0,102,personas,amigo,,,,,,femenino,,no,alimentos,0,1,0,0,,algunas_veces,no
1,103,personas,amigo,,,,,,masculino,,no,dinero alimentos,1,1,0,0,,algunas_veces,no
2,107,grupos,,,,,extranjeros,,,,no,alimentos,0,1,0,0,,algunas_veces,si
3,108,personas,familia,,otro_pariente,,,,,,no,alimentos,0,1,0,0,,algunas_veces,si
4,109,personas,vecino,,,,,,femenino,,si,alimentos,0,1,0,0,,una_vez,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,92,personas,familia,,hermanos,,,,femenino,,si,dinero alimentos,1,1,0,0,,frecuentemente,no
396,93,personas,familia,,padres,,,,femenino,,si,alimentos,0,1,0,0,,algunas_veces,si
397,96,personas,familia,,hermanos,,,,masculino,,si,alimentos,0,1,0,0,,algunas_veces,no
398,97,personas,familia,,otro_pariente,tío y primo,,,masculino,,si,dinero alimentos,1,1,0,0,,algunas_veces,no


In [77]:
for i in df_.columns[1:]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### huerto_relaciones

In [78]:
HUERTO_RELACIONES

,huerto_nombre_persona,_b_Sexo,huerto_relacion_social,huerto_otra_persona,huerto_parentesco,huerto_otro_pariente,huerto_tipo_relacion,huerto_tipo_relacion/interc_conocimiento,huerto_tipo_relacion/interc_produccion,huerto_tipo_relacion/interc_insumos,huerto_tipo_relacion/acopio_comercio,huerto_tipo_relacion/ensenanza,huerto_tipo_relacion/otra_relacion,huerto_otra_relacion,_2019_huerto_relacion,_2022_huerto_relacion,huertos_observaciones,_index,_parent_table_name,_parent_index
0,Invitan pero más en capacitaciones,femenino,trabajo,NaN,NaN,NaN,interc_conocimiento,1,0,0,0,0,0,NaN,si,si,Na,1,Vecindarios Alimentarios - Encuesta Integrada ...,2
1,Gente interesada en producir orgánicamente,masculino,otra_persona,Entidades y personas interesadas,NaN,NaN,interc_conocimiento interc_insumos otra_relacion,1,0,1,0,0,1,Gente de otros países interesados en la propue...,si,si,La gente de la zona muestra interés por ayudar...,2,Vecindarios Alimentarios - Encuesta Integrada ...,9
2,Hermanos de la iglesia de Jesucristo de los sa...,masculino,amigo,NaN,NaN,NaN,interc_conocimiento interc_produccion interc_i...,1,1,1,1,1,0,NaN,si,si,La iniciativa de la iglesia es prevenir en cas...,3,Vecindarios Alimentarios - Encuesta Integrada ...,10
3,Blanca Minango,femenino,vecino,NaN,NaN,NaN,interc_produccion,0,1,0,0,0,0,NaN,si,si,NaN,4,Vecindarios Alimentarios - Encuesta Integrada ...,364
4,En el mes de mayo se invita a conocer el huert...,NaN,otra_persona,Instituciones y vecinos de la comunidad,NaN,NaN,interc_conocimiento acopio_comercio ensenanza,1,0,0,1,1,0,NaN,si,si,NaN,5,Vecindarios Alimentarios - Encuesta Integrada ...,368
5,Huertos orgánicos que participan en bioferias,masculino,trabajo,NaN,NaN,NaN,interc_conocimiento ensenanza,1,0,0,0,1,0,NaN,si,si,"En conquito, agrupar hay los contactos",6,Vecindarios Alimentarios - Encuesta Integrada ...,371
6,Huertos orgánicos que participan en bioferias,masculino,trabajo,NaN,NaN,NaN,interc_conocimiento ensenanza,1,0,0,0,1,0,NaN,si,si,"En conquito, agrupar hay los contactos",7,Vecindarios Alimentarios - Encuesta Integrada ...,372
7,Huertos de vida Reino de Quito,masculino,vecino,NaN,NaN,NaN,interc_conocimiento interc_produccion,1,1,0,0,0,0,NaN,si,si,Es una asociación de vecinos para producir ali...,8,Vecindarios Alimentarios - Encuesta Integrada ...,375
8,Red de huertos Reino de Quito,masculino,otra_persona,Varios socios,NaN,NaN,interc_conocimiento interc_produccion interc_i...,1,1,1,0,0,0,NaN,si,si,Son parte de 120 socios solo 10 producen orgán...,9,Vecindarios Alimentarios - Encuesta Integrada ...,377


In [79]:
HUERTO_RELACIONES[HUERTO_RELACIONES['_parent_index'].duplicated()].size

0

In [80]:
df_ = HUERTO_RELACIONES.drop(columns=['_index','_parent_table_name'])
df_

,huerto_nombre_persona,_b_Sexo,huerto_relacion_social,huerto_otra_persona,huerto_parentesco,huerto_otro_pariente,huerto_tipo_relacion,huerto_tipo_relacion/interc_conocimiento,huerto_tipo_relacion/interc_produccion,huerto_tipo_relacion/interc_insumos,huerto_tipo_relacion/acopio_comercio,huerto_tipo_relacion/ensenanza,huerto_tipo_relacion/otra_relacion,huerto_otra_relacion,_2019_huerto_relacion,_2022_huerto_relacion,huertos_observaciones,_parent_index
0,Invitan pero más en capacitaciones,femenino,trabajo,NaN,NaN,NaN,interc_conocimiento,1,0,0,0,0,0,NaN,si,si,Na,2
1,Gente interesada en producir orgánicamente,masculino,otra_persona,Entidades y personas interesadas,NaN,NaN,interc_conocimiento interc_insumos otra_relacion,1,0,1,0,0,1,Gente de otros países interesados en la propue...,si,si,La gente de la zona muestra interés por ayudar...,9
2,Hermanos de la iglesia de Jesucristo de los sa...,masculino,amigo,NaN,NaN,NaN,interc_conocimiento interc_produccion interc_i...,1,1,1,1,1,0,NaN,si,si,La iniciativa de la iglesia es prevenir en cas...,10
3,Blanca Minango,femenino,vecino,NaN,NaN,NaN,interc_produccion,0,1,0,0,0,0,NaN,si,si,NaN,364
4,En el mes de mayo se invita a conocer el huert...,NaN,otra_persona,Instituciones y vecinos de la comunidad,NaN,NaN,interc_conocimiento acopio_comercio ensenanza,1,0,0,1,1,0,NaN,si,si,NaN,368
5,Huertos orgánicos que participan en bioferias,masculino,trabajo,NaN,NaN,NaN,interc_conocimiento ensenanza,1,0,0,0,1,0,NaN,si,si,"En conquito, agrupar hay los contactos",371
6,Huertos orgánicos que participan en bioferias,masculino,trabajo,NaN,NaN,NaN,interc_conocimiento ensenanza,1,0,0,0,1,0,NaN,si,si,"En conquito, agrupar hay los contactos",372
7,Huertos de vida Reino de Quito,masculino,vecino,NaN,NaN,NaN,interc_conocimiento interc_produccion,1,1,0,0,0,0,NaN,si,si,Es una asociación de vecinos para producir ali...,375
8,Red de huertos Reino de Quito,masculino,otra_persona,Varios socios,NaN,NaN,interc_conocimiento interc_produccion interc_i...,1,1,1,0,0,0,NaN,si,si,Son parte de 120 socios solo 10 producen orgán...,377


In [81]:
for i in df_.columns[:-1]:
    vecindarios[i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

In [82]:
all_profiles_cols = ['_1_1_codigo_cuestionario',
'_1_2_fecha_encuesta',
'_1_3_nombre_encuestador',
'_1_4_1_ciudad',
'_1_4_2_vecindario',
'sector_barrio_huerto',
'_1_5_perfil_entrevistado',
'_2_1_consentimiento_informado',
'_3_1_Nombre',
'_3_2_Informo_telefono',
'numero_telefono',
'_3_3_1_Distrito_Canton',
'_3_3_2_Provincia',
'_3_3_3_Pais',
'otro_pais',
'_3_6_nivel_estudio',
'nivel_completo',
'anos_nivel_incompleto',
'total_anos_estudio',
'total_miembros_hogar',
'miembro_parentesco',
'otro_pariente',
'otro_NO_pariente',
'sexo',
'edad_miembro',
'miembros_aporta_economicamente',
'_2019_perdio_empleo',
'_2020_perdio_empleo',
'_2020_tiempo_desempleado',
'_2020_mes_inicio_desempleado',
'_2020_mes_fin_desempleado',
'_2021_perdio_empleo',
'_2021_tiempo_desempleado',
'_2019_poca_variedad',
'_2019_porque_poca_variedad',
'_2020_poca_variedad',
'_2020_porque_poca_variedad',
'_2021_poca_variedad',
'_2021_porque_poca_variedad',
'_2019_poca_cantidad',
'_2019_porque_poca_cantidad',
'_2020_poca_cantidad',
'_2020_porque_poca_cantidad',
'_2021_poca_cantidad',
'_2021_porque_poca_cantidad',
'_2019_salto_comida',
'_2019_porque_salto_comida',
'_2020_salto_comida',
'_2020_porque_salto_comida',
'_2021_salto_comida',
'_2021_porque_salto_comida',
'_2019_no_comio_1_dia',
'_2019_porque_no_comio_1_dia',
'_2020_no_comio_1_dia',
'_2020_porque_no_comio_1_dia',
'_2021_no_comio_1_dia',
'_2021_porque_no_comio_1_dia',
'_1er_opcion',
'_2da_opcion',
'_3er_opcion',
'_4ta_opcion',
'_5ta_opcion',
'_6ta_opcion',
'_7ma_opcion',
'_8va_opcion',
'_9na_opcion',
'_10ma_opcion',
'otra_medida_para_tener_aliment',
'_9_1_recibio_apoyo',
'_a_de_quien_recibio_apoyo',
'personas_recibio_apoyo',
'otras_personas_recibio_apoyo',
'parentesco_recibio_apoyo',
'otros_parientes_recibio_apoyo',
'grupo_recibio_apoyo',
'organizacion_recibio_apoyo',
'_b_Sexo_recibio_apoyo',
'Nombre_grupo_organiz_recibio',
'_c_vecindario_recibio_apoyo',
'_d_tipo_apoyo_recibido',
'otro_tipo_apoyo_recibido',
'_e_frecuencia_recibio_apoyo',
'_f_Actualmente_recibe_apoyo',
'_10_1_padece_problemas_salud',
'otros_problemas_salud',
'problemas_paro',
'resolucion_problema_paro',
'datos_geo_referenciados',
'_datos_geo_referenciados_latitude',
'_datos_geo_referenciados_longitude',
'_datos_geo_referenciados_altitude',
'_datos_geo_referenciados_precision',
'Incluir_datos_georre_ciados_LATITUD_UTM',
'Incluir_datos_georre_iados_LONGITUD_UTM',]

In [102]:
vecindarios

Index(['_index', 'start', 'end', '_1_1_codigo_cuestionario',
       '_1_2_fecha_encuesta', '_1_3_nombre_encuestador', '_1_4_1_ciudad',
       '_1_4_2_vecindario', 'sector_barrio_huerto', '_1_5_perfil_entrevistado',
       ...
       'huerto_tipo_relacion/interc_conocimiento',
       'huerto_tipo_relacion/interc_produccion',
       'huerto_tipo_relacion/interc_insumos',
       'huerto_tipo_relacion/acopio_comercio',
       'huerto_tipo_relacion/ensenanza', 'huerto_tipo_relacion/otra_relacion',
       'huerto_otra_relacion', '_2019_huerto_relacion',
       '_2022_huerto_relacion', 'huertos_observaciones'],
      dtype='object', length=1050)

In [116]:
vecindarios['sexo'].isna().sum()==vecindarios.shape[0]

True

In [118]:
columns_to_drop = [i for i in vecindarios.columns if (vecindarios[i].isna().sum()==vecindarios.shape[0]) or (vecindarios[i].unique().size==1)]

In [120]:
vecindarios = vecindarios.drop(columns=columns_to_drop)

In [122]:
['miembro_parentesco', 'otro_pariente', 'otro_NO_pariente', 'sexo', 'edad_miembro', 'miembros_aporta_economicamente', '_10ma_opcion', '_a_de_quien_recibio_apoyo', 'personas_recibio_apoyo', 'otras_personas_recibio_apoyo', 'parentesco_recibio_apoyo', 'otros_parientes_recibio_apoyo', 'grupo_recibio_apoyo', 'organizacion_recibio_apoyo', '_b_Sexo_recibio_apoyo', 'Nombre_grupo_organiz_recibio', '_c_vecindario_recibio_apoyo', '_d_tipo_apoyo_recibido', 'otro_tipo_apoyo_recibido', '_e_frecuencia_recibio_apoyo', '_f_Actualmente_recibe_apoyo']

['miembro_parentesco',
 'otro_pariente',
 'otro_NO_pariente',
 'sexo',
 'edad_miembro',
 'miembros_aporta_economicamente',
 '_10ma_opcion',
 '_a_de_quien_recibio_apoyo',
 'personas_recibio_apoyo',
 'otras_personas_recibio_apoyo',
 'parentesco_recibio_apoyo',
 'otros_parientes_recibio_apoyo',
 'grupo_recibio_apoyo',
 'organizacion_recibio_apoyo',
 '_b_Sexo_recibio_apoyo',
 'Nombre_grupo_organiz_recibio',
 '_c_vecindario_recibio_apoyo',
 '_d_tipo_apoyo_recibido',
 'otro_tipo_apoyo_recibido',
 '_e_frecuencia_recibio_apoyo',
 '_f_Actualmente_recibe_apoyo']

In [125]:
vecindarios[vecindarios.columns.difference(all_profiles_cols)] = vecindarios[vecindarios.columns.difference(all_profiles_cols)].fillna(value='NO_VALIDO')

In [127]:
vecindarios

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,huerto_tipo_relacion/interc_conocimiento,huerto_tipo_relacion/interc_produccion,huerto_tipo_relacion/interc_insumos,huerto_tipo_relacion/acopio_comercio,huerto_tipo_relacion/ensenanza,huerto_tipo_relacion/otra_relacion,huerto_otra_relacion,_2019_huerto_relacion,_2022_huerto_relacion,huertos_observaciones
0,1,2022-07-29 22:39:16.546000,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO
1,2,2022-07-29 21:33:38.674000,2022-07-30 20:44:17.615999,NaN,2022-07-30,Fabian Sevilla,quito,solanda,NaN,productor,...,1.0,0.0,0.0,0.0,0.0,0.0,NO_VALIDO,si,si,Na
2,3,2022-08-03 14:34:00.904000,2022-08-03 15:25:01.405001,NaN,2022-08-03,Blanca Minango,quito,solanda,NaN,productor,...,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO
3,4,2022-08-05 08:45:32.981000,2022-08-05 09:43:39.466000,NaN,2022-08-05,Fabian Sevilla,quito,solanda,NaN,productor,...,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO
4,5,2022-08-05 10:17:44.338000,2022-08-05 10:55:58.182000,NaN,2022-08-05,Fabian Sevilla,quito,solanda,NaN,productor,...,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,891,2022-10-20 15:43:45.890999,2022-10-23 16:34:37.305000,1711429546,2022-10-20,Gabriela Paredes,quito,fuera,Asociación 24 de mayo,productor,...,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO
891,892,2022-10-23 18:08:27.725000,2022-10-23 19:14:02.357000,1802019800,2022-10-23,Gabriela Paredes,quito,fuera,Penileo - Píllaro,productor,...,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO
892,893,2022-10-29 16:40:56.251001,2022-10-29 17:02:32.614000,NaN,2022-08-18,Anny Diana Gomez de la Cruz,lima,carabayllo,NaN,productor,...,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO
893,894,2022-10-30 17:00:24.259000,2022-10-30 18:02:45.276000,NaN,2022-06-17,Jaime Perez Salinas,lima,fuera,no puso,productor,...,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO,NO_VALIDO


In [83]:
vecindarios_productor = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='productor'].reset_index(drop=True)

In [84]:
vecindarios_comerciante = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='comerciante'].reset_index(drop=True)

In [85]:
vecindarios_consumidor = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='consumidor'].reset_index(drop=True)

In [128]:
vecindarios.to_csv('final_version.csv', index=False)